In [1]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from tensorflow_probability import edward2 as ed

import tensorflow as tf

In [2]:
x, y = make_regression(n_samples=1000000, n_features=10, n_informative=7)

hparams = tf.contrib.training.HParams(test_size=0.2, batch_size=100, shuffle_iterations=1000, learning_rate=0.05,
                                     num_posterior_samples=1000)

In [3]:
def input_data(x, y, phase, hparams):
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=hparams.test_size)
    
    if phase == 'train':
        x_tensor = tf.convert_to_tensor(x_train, tf.float32)
        y_tensor = tf.convert_to_tensor(y_train, tf.float32)
        
        dataset = tf.data.Dataset.from_tensor_slices((x_tensor, y_tensor))
        shuffle = dataset.shuffle(hparams.shuffle_iterations)
        batches = shuffle.repeat().batch(hparams.batch_size)
        iterator = batches.make_one_shot_iterator()
        features, label = iterator.get_next()
        
        return features, label
    else:
        x_tensor = tf.convert_to_tensor(x_test, tf.float32)
        y_tensor = tf.convert_to_tensor(y_test, tf.float32)
        
        dataset = tf.data.Dataset.from_tensor_slices((x_tensor, y_tensor))
        shuffle = dataset.shuffle(hparams.shuffle_iterations)
        batches = shuffle.repeat().batch(hparams.batch_size)
        iterator = batches.make_one_shot_iterator()
        features, label = iterator.get_next()
        
        return features, label

In [4]:
class LinearModel(object):
    def __init__(self):
        self.hparams = hparams
        self.weights_dict = {'w':None, 'b':None}
    
    def fit(self, input_fn):
        
        features, label = input_fn()

        def linear_model(X): # (unmodeled) data
            w = ed.Normal(loc=tf.zeros([X.shape[1]]),
                scale=tf.ones([X.shape[1]]),
                name="w")  # parameter
            b = ed.Normal(loc=tf.zeros([]),
                scale=tf.ones([]), 
                name="b")  # parameter
            y = ed.Normal(loc=tf.tensordot(X, w, axes=1) + b, scale=1.0, 
                name="y")  # (modeled) data
            return y   

        def variational_model(qw_mean, qw_stddv, qb_mean, qb_stddv):
            qw = ed.Normal(loc=qw_mean, scale=qw_stddv, name="qw")
            qb = ed.Normal(loc=qb_mean, scale=qb_stddv, name="qb")          
            return qw, qb

        log_q = ed.make_log_joint_fn(variational_model)

        def target_q(qw, qb):
            return log_q(qw_mean=qw_mean, qw_stddv=qw_stddv, qb_mean=qb_mean, qb_stddv=qb_stddv, qw=qw, qb=qb)

        qw_mean = tf.Variable(tf.zeros([int(features.shape[1])]), dtype=tf.float32)
        qb_mean = tf.Variable(tf.zeros([]), dtype=tf.float32)
        qw_stddv = tf.nn.softplus(tf.Variable(tf.ones([int(features.shape[1])]), dtype=tf.float32))
        qb_stddv = tf.nn.softplus(tf.Variable(tf.ones([]), dtype=tf.float32))

        qw, qb = variational_model(qw_mean=qw_mean, qw_stddv=qw_stddv,
                                qb_mean=qb_mean, qb_stddv=qb_stddv)

        log_joint = ed.make_log_joint_fn(linear_model)

        def target(qw, qb):
            """Unnormalized target density as a function of the parameters."""
            return log_joint(w=qw, b=qb, X=features, y=label)

        energy = target(qw, qb) 
        entropy = -target_q(qw, qb) / hparams.batch_size
        elbo = energy + entropy

        optimizer = tf.train.AdamOptimizer(learning_rate = hparams.learning_rate)
        train = optimizer.minimize(-elbo)

        init = tf.global_variables_initializer()

        t = []
        n_rows = int(x.shape[0] * (1-hparams.test_size))
        num_steps = int(n_rows / hparams.batch_size)

        with tf.Session() as sess:
            sess.run(init)

            for step in range(num_steps):
                _ = sess.run([train])
                if step % 100 == 0:
                    t.append(sess.run([elbo]))
            
            self.weights_dict['w'], self.weights_dict['b'] = sess.run([qw.distribution.sample(hparams.num_posterior_samples), 
                                                             qb.distribution.sample(hparams.num_posterior_samples)])

    def evaluate(self, input_fn):

        features, label = input_fn()

        w_ = tf.reduce_mean(self.weights_dict['w'], 0)
        b_ = tf.reduce_mean(self.weights_dict['b'], 0)

        predictions = tf.cast((tf.tensordot(features, w_, axes=1) + b_), tf.float32)

        rmse, rmse_update_op = tf.metrics.root_mean_squared_error(tf.cast(label,tf.float32), predictions)

        n_rows = int(x.shape[0] * (hparams.test_size))
        n_batches = int(n_rows / hparams.batch_size)

        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            sess.run(tf.local_variables_initializer())
            for _i in range(n_batches):
                rmse_ = sess.run([rmse_update_op])

            metrics = {}
            metrics["rmse"] = sess.run([rmse])
            return metrics


In [5]:
predict_toy_regression = LinearModel()

In [6]:
train_input_fn = lambda:input_data(x, y, 'train', hparams)
test_input_fn = lambda:input_data(x, y, 'test', hparams)

In [7]:
predict_toy_regression.fit(train_input_fn)

/Users/allenchang/PycharmProjects/tf_probability/venv/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)


In [8]:
predict_toy_regression.evaluate(test_input_fn)

/Users/allenchang/PycharmProjects/tf_probability/venv/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)


{'rmse': [1.9667388]}